In [1]:
!pip install langchain-ollama
!pip install langchain-community

In [2]:
assistant_instructions = """ You are an internal sales assistant employed by our chatbot company, tasked with analyzing and extracting valuable insights from our sales calls. Your primary role is to assist in enhancing our sales call analysis based on past interactions. You are also responsible for various call-related tasks, including sentiment analysis, providing detailed responses to questions and queries, and ensuring the utmost accuracy in your responses.

Your source of information is a document file containing transcripts of our previous sales calls. Your goal is to leverage this data to offer comprehensive and insightful answers to a wide range of questions and inquiries. Your responses should go beyond basic information, aiming for depth and precision when addressing the queries.

Your assistance will play a pivotal role in improving our sales strategies and customer interactions. Your expertise in extracting insights from our sales calls will help us better understand customer needs, objections, and preferences, ultimately leading to more successful sales outcomes. """


In [3]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
import os


In [4]:
import pandas as pd

file_path = ('./data/sales10.csv') # insert the path of the csv file
data = pd.read_csv(file_path)

#preview the csv file
data.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [30]:
loader = CSVLoader(file_path=file_path)
docs = loader.load_and_split()
docs

[Document(metadata={'source': './data/sales10.csv', 'row': 0}, page_content='Invoice ID: 750-67-8428\nBranch: A\nCity: Yangon\nCustomer type: Member\nGender: Female\nProduct line: Health and beauty\nUnit price: 74.69\nQuantity: 7\nTax 5%: 26.1415\nTotal: 548.9715\nDate: 1/5/2019\nTime: 13:08\nPayment: Ewallet\ncogs: 522.83\ngross margin percentage: 4.761904762\ngross income: 26.1415\nRating: 9.1'),
 Document(metadata={'source': './data/sales10.csv', 'row': 1}, page_content='Invoice ID: 226-31-3081\nBranch: C\nCity: Naypyitaw\nCustomer type: Normal\nGender: Female\nProduct line: Electronic accessories\nUnit price: 15.28\nQuantity: 5\nTax 5%: 3.82\nTotal: 80.22\nDate: 3/8/2019\nTime: 10:29\nPayment: Cash\ncogs: 76.4\ngross margin percentage: 4.761904762\ngross income: 3.82\nRating: 9.6'),
 Document(metadata={'source': './data/sales10.csv', 'row': 2}, page_content='Invoice ID: 631-41-3108\nBranch: A\nCity: Yangon\nCustomer type: Normal\nGender: Male\nProduct line: Home and lifestyle\nUnit

In [16]:
from langchain_community.embeddings import OllamaEmbeddings 
#from langchain_ollama import OllamaEmbeddings
#from langchain_chroma import Chroma 
from langchain_community.chat_models import ChatOllama 
#from langchain_ollama import ChatOllama
#from langchain_ollama import OllamaLLM

embedding_model_name = "mxbai-embed-large"
#embedding_model_name = "llama3.2"
#embeddings = OllamaEmbeddings(model=embedding_model_name, show_progress=True)
#embeddings = OllamaEmbeddings(model=embedding_model_name)
embeddings = OllamaEmbeddings(model=embedding_model_name)

In [18]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings.embed_query(" ")))
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [19]:
vector_store.add_documents(documents=docs)

['a75166f8-78ed-42d6-bf18-44c93111e111',
 '9b410d9f-a666-4bb7-89b0-bbc6e6acf817',
 '8497675e-3d4e-42d2-99a8-fc48619321a7',
 'a388368d-49e7-4a2c-8198-79789c2c0e04',
 'fa736ddb-5d22-4953-a110-9b9cfdc16a0d',
 '46cb811f-2691-4ab0-bd78-474554e9583d',
 '264c73d7-bc3b-4ca1-be0f-6d1e9038fd7e',
 'a07cb312-360c-45ba-a99c-14d0eaf5bcb0',
 'cd2ea8c6-feb2-4350-8f4c-8405ab5366fc',
 'b2c6b613-a8e7-4888-acf6-dfb80b4d1cbb']

In [20]:
llm = ChatOllama(model="llama3.2:latest")

C:\Users\ramezania\AppData\Local\Temp\ipykernel_18892\1403308041.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3.2:latest")


In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

retriever = vector_store.as_retriever()

# Set up system prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
    
])


In [22]:
# Create the question-answer chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
retriever

In [23]:
answer= rag_chain.invoke({"input": "How much is total income?"})
answer

{'input': 'How much is total income?',
 'context': [Document(metadata={'source': './data/sales10.csv', 'row': 7}, page_content='Invoice ID: 315-22-5665\nBranch: C\nCity: Naypyitaw\nCustomer type: Normal\nGender: Female\nProduct line: Home and lifestyle\nUnit price: 73.56\nQuantity: 10\nTax 5%: 36.78\nTotal: 772.38\nDate: 2/24/2019\nTime: 11:38\nPayment: Ewallet\ncogs: 735.6\ngross margin percentage: 4.761904762\ngross income: 36.78\nRating: 8'),
  Document(metadata={'source': './data/sales10.csv', 'row': 1}, page_content='Invoice ID: 226-31-3081\nBranch: C\nCity: Naypyitaw\nCustomer type: Normal\nGender: Female\nProduct line: Electronic accessories\nUnit price: 15.28\nQuantity: 5\nTax 5%: 3.82\nTotal: 80.22\nDate: 3/8/2019\nTime: 10:29\nPayment: Cash\ncogs: 76.4\ngross margin percentage: 4.761904762\ngross income: 3.82\nRating: 9.6'),
  Document(metadata={'source': './data/sales10.csv', 'row': 2}, page_content='Invoice ID: 631-41-3108\nBranch: A\nCity: Yangon\nCustomer type: Normal\nGe

In [24]:
answer["answer"]

"To calculate the total income, I'll need to add up the gross incomes from each invoice: \n\n36.78 + 3.82 + 16.2155 + 29.8865 = 86.7825.\n\nSo, the total income is approximately $86.78."

In [25]:
answer= rag_chain.invoke({"input": "How much is sales total?"})
answer["answer"]

'The sales total is the sum of gross income from all invoices, which is: 3.82 + 36.78 + 3.626 + 26.1415 = 70.9865.'

In [28]:
answer= rag_chain.invoke({"input": "How much is sum of total of invoices?"})
answer["answer"]

'I can calculate the total amount from the provided invoice data. Here are the totals:\n\nInvoice ID: 631-41-3108 - Total: 340.5255\nInvoice ID: 226-31-3081 - Total: 80.22\nInvoice ID: 355-53-5943 - Total: 433.692\nInvoice ID: 665-32-9167 - Total: 76.146\n\nThe sum of the total amount from all invoices is approximately 990.0055.'

In [27]:
answer= rag_chain.invoke({"input": "How much is invoice totals?"})
answer["answer"]

'Invoice ID 315-22-5665 has a total of $772.38, and Invoice ID 631-41-3108 has a total of $340.5255. There are no other invoices provided with their total amounts listed.'

In [29]:
answer= rag_chain.invoke({"input": "How much is invoice totals?"})
answer["answer"]

'I can calculate the total for you.\n\nInvoice ID: 226-31-3081 has a total of $80.22.\nInvoice ID: 665-32-9167 has a total of $76.15.\nInvoice ID: 315-22-5665 has a total of $772.38.\nInvoice ID: 631-41-3108 has a total of $340.53.\n\nThese totals do not add up to a specific number, but the highest invoice is $772.38 for Invoice ID 315-22-5665.'